# The "Simple" Lesion Notebook

### Authors: Alexander Cohen and Louis Soussand - Version 1.6 - 12/27/2017
Changelog:

1.0 Initial Release

1.1 Added Code to allow choosing a subset of subjects by providing a textfile of IDs

1.2 Now able to receive dataset info from XNAT Gate and access the data by streaming XNAT backup on the Millenium Falcon.

1.3 Cleaned up cells  
1.3.1 Typo fixed

1.4 Fixed order of cells and 'indir' error
1.4.1 Fixed the selection of a Subject list

1.5 Added nimlab module (functions)

1.6 Added nimlab module (software)

#### This notebook is intendend to be run cell by cell to conduct lesion network analysis on a single set of lesions. Cells that contain #??? at the begininng have settings that you can/should modify.

Using this notebook, you can:

1) Load necessary packages

2) Select the input and output directories
(This notebook expects the input to have subdirectories of "Lesions" and "Functional_Connectivity" with the connectome.sh output in the latter.)

3) Explore the lesions visually

4) Review the amount of overlap for a range of thresholds with the fc T-maps

5) Choose a specific threshold and review the spatial extent of the overlap at various 'fractional' thresholds (# subjects/total)

6) Finalize a figure and NIFTI file of the peak locations

7) Use this to generate ROIs

8) Obtain the correlations between these ROIs and each subjects' lesion

9) Display the results as a series of boxplots

For more detailed analysis, switch to the Two-Sample T-Test or ROI Predictor notebook.

In [ ]:
## Packages and environmental settings:

## Environment:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

##Packages:
import sys
import string
import os
import glob
import math
import numpy as np
import hdf5storage as hdf
import scipy as sp
import pandas as pd
import nilearn
import nibabel as nb
from nilearn import plotting, image, masking, regions
import io
import time
import subprocess
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import matplotlib.image as mpimg

import multiprocessing

from time import time
from random import random

## nimlab package is hosted internally on the M Falcon
sys.path.append('/data2/jupyter/notebooks/modules/')
from nimlab.functions import *
from nimlab.software import *

## Where are the lesion and functional connectivity files and where do you want your output?

In [ ]:
#??? Inputs Required:

# ------------------------- XNAT_Gate Datasets ----------------------
# Set XNAT_file='' if not using an XNAT csv file:
XNAT_file='Prosopagnosia.csv'
# XNAT_file=''

if XNAT_file:
    # Target (this is just a label to distinguish this analysis from another analysis)
    Target='PA_nonFFA_Patients'
    # Set this to where you would like the outputs to be written
    output_folder="/data2/jupyter/notebooks/acohen8/PA/xnat_notebook_analysis"
    
    # Don't touch these four lines unless you understand the code.
    filename=XNAT_file.replace('.csv', '')
    path='XNAT Dataset'
    Dataset=filename
#     Subject_List=''
    Subject_List=''
# ------------------------- XNAT_Gate Datasets ----------------------



# ------------------------- Local Datasets --------------------------
if not XNAT_file:
    # Set this to the location of your target data, with sub-folders of: "Lesions" and "Functional_Connectivity":
    path="/home/acohen8/projects/PA"

    # Set this to where you would like the outputs to be written (the output below will be put into a sub-directory here)
    output_folder="/home/acohen8/projects/PA/notebook_analysis"

    # Dataset:
    Dataset='Prosopagnosia'

    # List of Specific Subject IDs you want to examine (set to '' if you want everyone):
    Subject_List='/home/acohen8/projects/PA/notebook_analysis/Prosopagnosia_Cases_output_directory/ROIs/Prosopagnosia_Cases_FC_Peaks_T9_43of44_ROI0_exclusion_list.txt'
    # Subject_List=''

    # Target (this is just a label to distinguish this analysis from another analysis)
    Target='Non_FFA'
# ------------------------- Local Datasets --------------------------

### Get the Lesion and FC Filenames for the Chosen Dataset:

In [ ]:
# Load in the file names

indir=path+'/'
if not XNAT_file:
    # Get Lesion, fc_T Files, and fc_Fz Files
    TF = sorted(glob.glob(indir+'Lesions/'+"*gz"))
    lcs=longest_common_suffix(TF)

    Target_IDs=[]
    for i in range(0, len(TF)):
        sub=os.path.basename(TF[i][0:-len(lcs)])
        Target_IDs.append(sub)

    # Selection of Specific Subjects
    if Subject_List:
        f = open(Subject_List, 'r')
        Target_IDs = f.read().splitlines()
    N_of_images=len(Target_IDs)

    # Get Lesion, fc_T Files, and fc_Fz Files
    Target_files = [glob.glob(indir+'Lesions/'+str(entry)+"*gz")[0] for entry in Target_IDs]
    Target_fc_files = [glob.glob(indir+'Functional_Connectivity/'+str(entry)+"*_func_seed_T.nii.gz")[0] for entry in Target_IDs]
    Target_fc_Fz_files = [glob.glob(indir+'Functional_Connectivity/'+str(entry)+"*_func_seed_AvgR_Fz.nii.gz")[0] for entry in Target_IDs]    
    
else:    
    #Read csv file in pandas dataframe format:
    pd_df=pd.read_csv(XNAT_file)

    Target_IDs=pd_df['subject']

    # Selection of Specific Subjects
    if Subject_List:
        f = open(Subject_List, 'r')
        Target_IDs = f.read().splitlines()
        pd_df = pd_df.loc[pd_df['subject'].isin(Target_IDs)]
        pd_df.index = pd.RangeIndex(len(pd_df.index))
    N_of_images=len(Target_IDs)

    # Get Lesion, fc_T Files, and fc_Fz Files
    Target_files = pd_df['Lesion']
    Target_fc_files = pd_df['func_T']
    Target_fc_Fz_files = pd_df['AvgR_Fz']

## Output Control:

In [ ]:
## Output control:

outdir=output_folder+'/'+Dataset+'_'+Target+'_output/'
    
if not os.path.exists(outdir):
    #create the output directory
    os.makedirs(outdir)
   
    #Within the output directory, 3 subfolders are created:
   
    #output images
if not os.path.exists(outdir+'images/'):
    os.makedirs(outdir+'images/')
    #final ROIs (not lesions)
if not os.path.exists(outdir+'ROIs/'):
    os.makedirs(outdir+'ROIs/')
    #Analyses: empty subfolder for subsequent analyses
if not os.path.exists(outdir+'Analyses/'):
    os.makedirs(outdir+'Analyses/')

### Print Selections:

In [ ]:
print 'The input directory being used is: '+ path
print 'The output directory being used is: ' + output_folder

print 'The dataset label is: '+ Dataset
print 'The symptom we are going to study is: '+ Target

### The following cell provide visual display of the underlying lesion data, thus does not need to be run more than once per dataset.

In [ ]:
# Lesion Visualization:
Target_lesions = [image.load_img(entry) for entry in Target_files]
Target_lesion_overlap = image.math_img("img*"+str(len(Target_files)),img=image.mean_img(Target_lesions))
lesion_mask = image.math_img("img>=1",img=Target_lesion_overlap)

fast_glass_brain_plot(Target_lesion_overlap,colorbar=True,display_mode='lyrz',title=" N="+str(N_of_images),output_file=outdir+"images/"+Dataset+"_"+Target+"_Lesions_glass.png")
fidl_plot2(Target_lesion_overlap,"N="+str(N_of_images),filename_to_use=outdir+"images/"+Dataset+"_"+Target+"_Lesions_slices.png")

fast_glass_brain_plot(lesion_mask,colorbar=True,display_mode='lyrz',cmap="Reds",title="N="+str(N_of_images)+" Coverage",output_file=outdir+"images/"+Dataset+"_"+Target+"_Lesions_coverage_glass.png")
fidl_plot2(lesion_mask,"N="+str(N_of_images)+" Coverage",cmap_to_use="Reds",filename_to_use=outdir+"images/"+Dataset+"_"+Target+"_Lesions_coverage_slices.png")

In [ ]:
# Create folder for list of files:
# Make text files for each lists of cases and controls

#Cases:
Target_lesion_filelist = open(outdir+'/'+Dataset+"_"+Target+'_cases_lesion_files.txt','w+')
for entry in Target_files:
    print>>Target_lesion_filelist,entry
Target_lesion_filelist.close()

Target_fc_filelist = open(outdir+'/'+Dataset+"_"+Target+'_cases_fcT_files.txt','w+')
for entry in Target_fc_files:
    print>>Target_fc_filelist,entry
Target_fc_filelist.close()

Target_fc_Fz_filelist = open(outdir+'/'+Dataset+"_"+Target+'_cases_Fz_files.txt','w+')
for entry in Target_fc_Fz_files:
    print>>Target_fc_Fz_filelist,entry
Target_fc_Fz_filelist.close()

### The following cell provide visual display of each lesion and its corresponding fc T map, thus does not need to be run more than once per dataset.

In [ ]:
# Show each individual lesion and connectivity (note, this cell does all of the work, and THEN shows you the results)

for idx, entry in enumerate(Target_IDs):
    print "Working on "+str(entry)
    plotting.plot_stat_map(Target_files[idx],colorbar=False,title=str(entry),cmap="bwr",draw_cross=False)
    fast_glass_brain_plot(Target_files[idx], display_mode='lyrz', title=str(entry),colorbar=False)
    fast_glass_brain_plot(Target_fc_files[idx],plot_abs=False,colorbar=True,display_mode='lyrz',title=str(entry))

# Start of Actual Lesion Network Analysis

## Find maximal functional connectivity T-map Network Overlaps

In [ ]:
# Load up T maps
Target_lesion_fc_networks = [image.load_img(entry) for entry in Target_fc_files]
print Target+" N="+str(len(Target_IDs))

In [ ]:
#??? Inputs Required: 

# This part can take 10-15 minutes
# Threshold and compute overlap 
# Set T_levels to a range of T-values to examine. 4-17, which in python is range(4,18,1) seems a reasonable range
# for the Yeo 1000 dataset, but feel free to change it.
T_levels=range(4,18,1)

n=str(len(Target_files))
start=time()
for i in T_levels:
    Target_lesion_fc_networks_thresholded_pos = [image.math_img('img > '+str(i), img=entry) for entry in Target_lesion_fc_networks]
    Target_lesion_fc_network_thresholded_pos_overlap = image.math_img("img*"+n,img=image.mean_img(Target_lesion_fc_networks_thresholded_pos))
    fidl_plot2(Target_lesion_fc_network_thresholded_pos_overlap,Target+" FC Pos Overlap, Threshold=" + str(i), filename_to_use=outdir+"images/"+Dataset+"_"+Target+"_case_FC_Pos_Overlap_Threshold"+str(i)+"_slices.png")
    fast_glass_brain_plot(Target_lesion_fc_network_thresholded_pos_overlap,plot_abs=False,colorbar=True,display_mode='lyrz', title=Target+" FC Pos Overlap, Threshold="+str(i),cmap=hotBlues(),output_file=outdir+"images/"+Dataset+"_"+ Target+"_case_FC_Pos_Overlap_Threshold"+str(i)+"_glass.png")

    Target_lesion_fc_networks_thresholded_neg = [image.math_img('img < -'+str(i), img=entry) for entry in Target_lesion_fc_networks]
    Target_lesion_fc_network_thresholded_neg_overlap = image.math_img("img*"+n,img=image.mean_img(Target_lesion_fc_networks_thresholded_neg))
    fidl_plot2(Target_lesion_fc_network_thresholded_neg_overlap,Target+" FC Neg Overlap, Threshold="+str(i),filename_to_use=outdir+"images/"+Dataset+"_"+Target+"_case_FC_Neg_Overlap_Threshold"+str(i)+"_slices.png",cmap_to_use=hotBlues(reverse=True))
    fast_glass_brain_plot(Target_lesion_fc_network_thresholded_neg_overlap,plot_abs=False,colorbar=True,display_mode='lyrz',title=Target+" FC Neg Overlap, Threshold="+str(i),cmap=hotBlues(reverse=True),output_file=outdir+"images/"+Dataset+"_"+Target+"_case_FC_Neg_Overlap_Threshold"+str(i)+"_glass.png")    
end=time()

print end-start    

In [ ]:
# show me a curve of maximal overlap vs FC T-map Tscore threshold

max_pos_overlap=[0]*len(T_levels)
max_neg_overlap=[0]*len(T_levels)
for idx,i in enumerate(T_levels):
    Target_lesion_fc_networks_thresholded_pos = [image.math_img('img > '+str(i), img=entry) for entry in Target_lesion_fc_networks]
    Target_lesion_fc_network_thresholded_pos_overlap = image.math_img("img*"+n,img=image.mean_img(Target_lesion_fc_networks_thresholded_pos))
    Target_lesion_fc_networks_thresholded_neg = [image.math_img('img < -'+str(i), img=entry) for entry in Target_lesion_fc_networks]
    Target_lesion_fc_network_thresholded_neg_overlap = image.math_img("img*"+n,img=image.mean_img(Target_lesion_fc_networks_thresholded_neg))
    
    max_pos_overlap[idx]=np.max(Target_lesion_fc_network_thresholded_pos_overlap.get_data())
    max_neg_overlap[idx]=np.max(Target_lesion_fc_network_thresholded_neg_overlap.get_data())

plt.plot(T_levels,max_pos_overlap,linewidth=3,label='Positive')
plt.plot(T_levels,max_neg_overlap,linewidth=3,label='Negative')
plt.title("Max overlap vs T-score cutoff")
plt.xlabel("T-score cutoff")
plt.ylabel("Max overlap")
plt.legend();

## Using a specific threshold looking for amount of overlap

In [ ]:
#??? Inputs Required: 

# Set i to the chosen T-score threshold you would like to use to binarize the maps prior to 
# calculating the amount of overlap.
T_level=10

# Set thres_maps_range to the number of subjects' overlap you want to threshold the map at.
# starting at 80% of your sample up to the max is not a bad place to start, but you're interested in the top end.
thres_maps_range=range(int(.8*len(Target_files)),len(Target_files)+1,1)
# thres_maps_range=range(30,45,1)

i=T_level
n=str(len(Target_files))
start=time()
Target_lesion_fc_networks_thresholded_pos = [image.math_img('img > '+str(i), img=entry) for entry in Target_lesion_fc_networks]
Target_lesion_fc_network_thresholded_pos_overlap = image.math_img("img*"+n,img=image.mean_img(Target_lesion_fc_networks_thresholded_pos))
Target_lesion_fc_networks_thresholded_neg = [image.math_img('img < -'+str(i), img=entry) for entry in Target_lesion_fc_networks]
Target_lesion_fc_network_thresholded_neg_overlap = image.math_img("img*"+n,img=image.mean_img(Target_lesion_fc_networks_thresholded_neg))

for thres_maps in thres_maps_range:
    fidl_plot2(image.threshold_img(Target_lesion_fc_network_thresholded_pos_overlap,threshold=thres_maps),Target+" FC Pos Overlap, Thres="+str(i)+"Frac="+str(thres_maps),filename_to_use=outdir+"images/"+Dataset+"_"+Target+"_case_FC_Pos_Overlap_Threshold"+str(i)+"_Frac"+str(thres_maps)+"_slices.png")
    fast_glass_brain_plot(image.threshold_img(Target_lesion_fc_network_thresholded_pos_overlap,threshold=thres_maps),plot_abs=False,colorbar=True,display_mode='lyrz',title=Target+" FC Pos Overlap, Threshold="+str(i)+"Fraction"+str(thres_maps),cmap=hotBlues(),output_file=outdir+"images/"+Dataset+"_"+Target+"_case_FC_Pos_Overlap_Threshold"+str(i)+"_Frac"+str(thres_maps)+"_glass.png")
    
    fidl_plot2(image.threshold_img(Target_lesion_fc_network_thresholded_neg_overlap,threshold=thres_maps),Target+" FC Neg Overlap, Thres="+str(i)+"Frac="+str(thres_maps),filename_to_use=outdir+"images/"+Dataset+"_"+Target+"_case_FC_Neg_Overlap_Threshold"+str(i)+"_Frac"+str(thres_maps)+"_slices.png",cmap_to_use=hotBlues(reverse=True))
    fast_glass_brain_plot(image.threshold_img(Target_lesion_fc_network_thresholded_neg_overlap,threshold=thres_maps),plot_abs=False,colorbar=True,display_mode='lyrz',title=Target+" FC Neg Overlap, Threshold="+str(i)+"Fraction"+str(thres_maps),cmap=hotBlues(reverse=True),output_file=outdir+"images/"+Dataset+"_"+Target+"_case_FC_Neg_Overlap_Threshold"+str(i)+"_Frac"+str(thres_maps)+"_glass.png")
end=time()
print end-start

In [ ]:
#??? Inputs Required: 

# Using a specific threshold and specific fractions, show me the peaks.
# Typically you want pos and neg to be the same, but if needed, you can set them separately:
T_level=10
pos_frac=18
neg_frac=18
n=str(len(Target_files))

Target_lesion_fc_networks_thresholded_pos = [image.math_img('img > '+str(T_level), img=entry) for entry in Target_lesion_fc_networks]
Target_lesion_fc_network_thresholded_pos_overlap = image.math_img("img*"+n,img=image.mean_img(Target_lesion_fc_networks_thresholded_pos))

Target_lesion_fc_networks_thresholded_neg = [image.math_img('img < -'+str(T_level), img=entry) for entry in Target_lesion_fc_networks]
Target_lesion_fc_network_thresholded_neg_overlap = image.math_img("img*"+n,img=image.mean_img(Target_lesion_fc_networks_thresholded_neg))

Target_lesion_fc_network_thresholded_map = image.math_img("img1 - img2",img1=image.threshold_img(Target_lesion_fc_network_thresholded_pos_overlap,threshold=pos_frac),img2=image.threshold_img(Target_lesion_fc_network_thresholded_neg_overlap,threshold=neg_frac))

fidl_plot2(Target_lesion_fc_network_thresholded_map,Target+" FC Peaks", filename_to_use=outdir+"images/"+Dataset+"_"+Target+"_case_FC_Peaks_slices.png",cmap_to_use=hotBlues())
fast_glass_brain_plot(Target_lesion_fc_network_thresholded_map,plot_abs=False,colorbar=True,display_mode='lyrz',title=Target+" FC Peaks",cmap=hotBlues(),output_file=outdir+"images/"+Dataset+"_"+Target+"_case_FC_Peaks_glass.png")

## Obtain ROIs and compare Lesion-ROI connectivity scores

In [ ]:
# Print the peaks image and the underlying overlap map to nifti files:

Target_lesion_fc_network_underlying_map = image.math_img("img1 - img2",img1=Target_lesion_fc_network_thresholded_pos_overlap,img2=Target_lesion_fc_network_thresholded_neg_overlap)

Target_lesion_fc_network_underlying_map.to_filename(outdir+"ROIs/"+Dataset+"_"+Target+"_FC_Peaks_T"+str(T_level)+".nii.gz")
Target_lesion_fc_network_thresholded_map.to_filename(outdir+"ROIs/"+Dataset+"_"+Target+"_FC_Peaks_T"+str(T_level)+"_"+str(pos_frac)+"of"+n+".nii.gz")

In [ ]:
#??? Inputs Required: 

# You may need to play with this to help the peak detector produce the regions you want,
# this is in mm^3, for small ditzels, try 10, for bigger blobs try 500, etc...
minimum_region_size=10

# Isolate the regions into separate ROIs:
peak_regions, roi_index = regions.connected_regions(Target_lesion_fc_network_thresholded_map,extract_type='connected_components',min_region_size=minimum_region_size)

In [ ]:
# Show me each ROI

a=range(0,len(roi_index))
for i in a:
    img = image.index_img(peak_regions,i)
    plotting.plot_stat_map(img,colorbar=False,title="ROI #"+str(i),cmap="bwr")

In [ ]:
#??? Inputs Required: 

# Print out the regions you want as nifti files:
# The numbers here correspond to the legends in the figure above (i.e., starts with #0)
rois_to_print=[0, 5]

for i in rois_to_print:
    image.math_img("img!=0",img=image.index_img(peak_regions,i)).to_filename(outdir+"ROIs/"+Dataset+"_"+Target+"_FC_Peaks_T"+str(T_level)+"_"+str(pos_frac)+"of"+n+"_ROI"+str(i)+".nii.gz")

## Additional Analyses start here:

In [ ]:
# make a text file list of the ROIs with absolute paths
list_of_ROIs=sorted(glob.glob(outdir+"ROIs/"+Dataset+"_"+Target+"_FC_Peaks_T"+str(T_level)+"_"+str(pos_frac)+"of"+n+"_ROI*.nii.gz"))
list_of_ROIs_filelist=open(outdir+"Analyses/"+Dataset+"_"+Target+"_FC_Peaks_T"+str(T_level)+"_"+str(pos_frac)+"of"+n+"_ROI_list.txt",'w+')
for entry in list_of_ROIs:
    print>>list_of_ROIs_filelist,os.path.abspath(entry)
list_of_ROIs_filelist.close()

ROI_names=[entry[-11:-7] for entry in list_of_ROIs]

In [ ]:
os.path.basename(Target_files[0])

In [ ]:
# Load List of ROI of interest and List of Lesion ROIs, then run against Yeo and save correl values

# connectome.sh requires .nii files, for XNAT datasets, 
# a nii copy of the lesion files is created in ROIs/niiLesions

# This takes a while, watch top on the terminal or follow connectome.log

ROI_vs_lesion_list=open(outdir+"Analyses/"+"Current_lesion_vs_ROI_list.txt",'w+')
for entry in list_of_ROIs:
    print>>ROI_vs_lesion_list,os.path.abspath(entry)

if XNAT_file:
    Target_files_nii=[]
    if not os.path.exists(outdir+'ROIs/niiLesions'):
        os.makedirs(outdir+'ROIs/niiLesions')
    for entry in Target_files:
        o=outdir+'ROIs/niiLesions'+'/'+os.path.basename(entry)[:-3]
        Target_files_nii.append(o)
        if not os.path.exists(o):
            !gunzip -c {entry} > {o}
        print>>ROI_vs_lesion_list,o
else:
    for entry in Target_files:
        print>>ROI_vs_lesion_list,entry
ROI_vs_lesion_list.close()



In [ ]:
read_input=outdir+"Analyses/"+"Current_lesion_vs_ROI_list.txt"
output_directory=outdir+"Analyses/"

call_connectome(read_input=read_input, output_directory=output_directory, numWorkers=20)

In [ ]:
# Load up the results and check the data (note you only want the first couple of columns, etc...)
ROI_lesion_results_file=hdf.loadmat(outdir+"Analyses/matrix_corrMx_AvgR.mat")
ROI_lesion_results_Tfile=hdf.loadmat(outdir+"Analyses/matrix_corrMx_T.mat")

plt.imshow(ROI_lesion_results_file['X'], interpolation='nearest')
plt.show()

In [ ]:
# Show me the correlation from the ROIs to the subjects's lesions. This is just a rough example, for a more complete version, use the ROI_Binary_Predictor Notebook

from scipy import stats

# Show me the correlation from the ROIs to the subjects's lesions:

ROI_to_lesion_corr=ROI_lesion_results_file['X'][0,len(list_of_ROIs):]
ROI_to_lesion_corr2=ROI_lesion_results_file['X'][1,len(list_of_ROIs):]

ROI_to_lesion_T=ROI_lesion_results_Tfile['X'][0,len(list_of_ROIs):]
ROI_to_lesion_T2=ROI_lesion_results_Tfile['X'][1,len(list_of_ROIs):]

# plt.boxplot([ROI_to_lesion_corr, ROI_to_lesion_corr2])
# plt.boxplot([ROI_to_lesion_T, ROI_to_lesion_T2])

plt.subplot(1, 2, 1)
plt.boxplot([ROI_to_lesion_corr, ROI_to_lesion_corr2])
plt.subplot(1, 2, 2)
plt.boxplot([ROI_to_lesion_T, ROI_to_lesion_T2])
plt.show()